# Unxt

Author: Nathaniel Starkman (MIT, 2025)

With `quax` we can do units in JAX!

In [1]:
import jax
import jax.numpy as jnp
from beartype import beartype
from jaxtyping import Float

Why doesn't Astropy work out of the box?

In [2]:
import astropy.units as u

In [3]:
x = u.Quantity([1, 2, 3], "m")

In [4]:
jnp.sqrt(x)

Array([1.       , 1.4142135, 1.7320508], dtype=float32)

We lost the units!

Astropy arrays are converted to NumPy arrays, which are then converted to JAX arrays.

Let's try the `quaxed` version.

In [5]:
import quaxed.numpy as qnp

In [6]:
qnp.sqrt(x)

Array([1.       , 1.4142135, 1.7320508], dtype=float32)

Darn. same problem.

Introducing...

## Quantity

In [7]:
from unxt import Quantity
from unxt import uconvert, ustrip  # alt functional API

In [8]:
x = Quantity([1, 2, 3], 'm')
x

Quantity(Array([1, 2, 3], dtype=int32), unit='m')

In [9]:
try:
    jnp.sqrt(x)
except TypeError as e:
    print(e)

sqrt requires ndarray or scalar arguments, got <class 'unxt._src.quantity.quantity.Quantity[PhysicalType('length')]'> at position 0.


In [10]:
from quax import quaxify

quaxify(jnp.sqrt)(x)

Quantity(Array([1.       , 1.4142135, 1.7320508], dtype=float32), unit='m(1/2)')

In [11]:
# Alternatively, use the `quaxed` version.
qnp.sqrt(x)

Quantity(Array([1.       , 1.4142135, 1.7320508], dtype=float32), unit='m(1/2)')

Now let's return to the Astropy Quantity

In [12]:
qnp.sqrt(x)

Quantity(Array([1.       , 1.4142135, 1.7320508], dtype=float32), unit='m(1/2)')

It worked?!? What happened?

Well, `unxt` hooked into `quax` and said it knows how to handle Astropy Quantities.
That's some nice interoperability.

### Astropy-like API

In [13]:
x.to(u.km)

Quantity(Array([0.001, 0.002, 0.003], dtype=float32, weak_type=True), unit='km')

In [14]:
x.to_value(u.km)

Array([0.001, 0.002, 0.003], dtype=float32, weak_type=True)

In [15]:
x.decompose([u.imperial.furlong, u.Gyr])

Quantity(Array([0.00497097, 0.00994194, 0.01491291], dtype=float32, weak_type=True),
    unit='fur')

### Parametric

`unxt.Quantity` can be parametrized by its dimensions!
This can be run-time inferred or set and used for verification

In [16]:
Quantity["frequency"](1, 1/u.s)  # works

Quantity(Array(1, dtype=int32, weak_type=True), unit='1 / s')

In [17]:
try:
    Quantity["frequency"](1, "kpc")
except Exception as e:
    print(e)

Physical type mismatch.


In [18]:
@beartype
def func(x: Quantity["length"], v: Quantity["speed"]) -> Quantity["time"]:
    return x / v


In [19]:
func(Quantity(1, "kpc"), Quantity(100, "km/s"))

Quantity(Array(0.01, dtype=float32, weak_type=True), unit='kpc s / km')

In [20]:
try:
    func(Quantity(1, "kpc"), Quantity(100, "1/s"))
except Exception as e:
    print(e)

Function __main__.func() parameter v="Quantity(Array(100, dtype=int32, weak_type=True), unit='1 / s')" violates type hint <class 'unxt._src.quantity.quantity.Quantity[PhysicalType({'speed', 'velocity'})]'>, as <class "unxt._src.quantity.quantity.Quantity[PhysicalType('frequency')]"> "Quantity(Array(100, dtype=int32, weak_type=True), unit='1 / s')" not instance of <class "unxt._src.quantity.quantity.Quantity[PhysicalType({'speed', 'velocity'})]">.


### Constructor

`unxt` offers powerful methods to construct a `Quantity`, through a generalized constructor class-method.

In [21]:
# normal construction
Quantity.from_([1, 2, 3], 'm')

Quantity(Array([1, 2, 3], dtype=int32), unit='m')

In [22]:
# From a Quantity
Quantity.from_(x)

Quantity(Array([1, 2, 3], dtype=int32), unit='m')

In [23]:
# Also changing the unit
Quantity.from_(x, "km")

Quantity(Array([0.001, 0.002, 0.003], dtype=float32, weak_type=True), unit='km')

In [24]:
Quantity.from_._f.methods

List of 9 method(s):
    [0] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value: typing.Union[ArrayLike,                  
    list[typing.Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''], numpy.bool, numpy.number,        
    bool, int, float, complex]], tuple[typing.Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''],    
    numpy.bool, numpy.number, bool, int, float, complex], ...]], unit: Any, *, dtype) ->                           
    unxt._src.quantity.base.AbstractQuantity                                                                       
        <function from_ at 0x116d8f6a0> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:799                  
    [1] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value: typing.Union[ArrayLike,                  
    list[typing.Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''], numpy.bool, numpy.number,        
    bool, int, float, complex]], tuple[typing.Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''],    
    numpy.bool, numpy.number, bool, int, float, complex], ...]], *, unit, dtype) ->                                
    unxt._src.quantity.base.AbstractQuantity                                                                       
        <function from_ at 0x116d8f9c0> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:838                  
    [2] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], *, value, unit, dtype) ->                       
    unxt._src.quantity.base.AbstractQuantity                                                                       
        <function from_ at 0x116d8fce0> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:863                  
    [3] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], mapping: collections.abc.Mapping[str,           
    typing.Any]) -> unxt._src.quantity.base.AbstractQuantity                                                       
        <function from_ at 0x116dac220> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:883                  
    [4] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value:                                          
    unxt._src.quantity.base.AbstractQuantity, unit: Any, *, dtype) -> unxt._src.quantity.base.AbstractQuantity     
        <function from_ at 0x116dac360> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:909                  
    [5] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value:                                          
    unxt._src.quantity.base.AbstractQuantity, unit: NoneType, *, dtype) ->                                         
    unxt._src.quantity.base.AbstractQuantity                                                                       
        <function from_ at 0x116dac680> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/jaxtyping/_decorator.py:935                  
    [6] from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value:                                          
    unxt._src.quantity.base.AbstractQuantity, *, unit, dtype) -> unxt._src.quantity.base.AbstractQuantity          
        <function from_ at 0x116dac9a0> @ ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and      
    Flatiron/Flatiron Presentation/

In [25]:
# Using a dictionary
Quantity.from_({"value": x, "unit": "km"})

Quantity(Array([0.001, 0.002, 0.003], dtype=float32, weak_type=True), unit='km')

In [26]:
Quantity.from_?

Signature: Quantity.from_(*args: Any, **kwargs: Any) -> 'AbstractQuantity'
Docstring:
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

from_(cls: type[unxt._src.quantity.base.AbstractQuantity], value: Union[ArrayLike, list[Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''], numpy.bool, numpy.number, bool, int, float, complex]], tuple[Union[jaxtyping.Shaped[Array, ''], jaxtyping.Shaped[ndarray, ''], numpy.bool, numpy.number, bool, int, float, complex], ...]], unit: Any, /, *, dtype: Any = None) -> unxt._src.quantity.base.AbstractQuantity

Construct a `unxt.Quantity` from an array-like value and a unit.

:p

## Non-Parametric

Parametric types can be slower (it's pretty marginal).
If in a hot-path it ends up being a limiting factor,
`unxt` offers a non-parametric Quantity.

In [27]:
from unxt.quantity import BareQuantity

In [28]:
x = BareQuantity(100, "km")
x

BareQuantity(Array(100, dtype=int32, weak_type=True), unit='km')

In [29]:
qnp.cbrt(x)

BareQuantity(Array(4.641589, dtype=float32, weak_type=True), unit='km(1/3)')

## Distance / Parallax

This is similar to Astropy's 
``astropy.coordinates.Distance``

In [30]:
from coordinax import Distance

In [31]:
d = Distance([1, 2, 3], 'm')
d

Distance(Array([1, 2, 3], dtype=int32), unit='m')

We can use it as normal

In [32]:
qnp.sqrt(d)

BareQuantity(Array([1.       , 1.4142135, 1.7320508], dtype=float32), unit='m(1/2)')

Like Astropy, this has methods for converting between distance measures

In [33]:
d.distance

Distance(Array([1, 2, 3], dtype=int32), unit='m')

In [34]:
d.parallax

Parallax(Array([1.5707964, 1.5707964, 1.5707964], dtype=float32), unit='rad')

In [35]:
d.distance_modulus

DistanceModulus(Array([-87.44675 , -85.941605, -85.06115 ], dtype=float32), unit='mag')

There are convenient constructors

Parallax:

In [36]:
Distance.from_(Quantity([1.], "rad"))

Distance(Array([0.6420926], dtype=float32), unit='AU')

See the Parallax class

In [37]:
from coordinax.angle import Parallax

In [38]:
p = Parallax([1., 2, 3], 'mas')
p

Parallax(Array([1., 2., 3.], dtype=float32), unit='mas')

In [39]:
p.distance

Distance(Array([2.0626480e+08, 1.0313240e+08, 6.8754936e+07], dtype=float32), unit='AU')

In [40]:
p.parallax

Parallax(Array([1., 2., 3.], dtype=float32), unit='mas')

In [41]:
p.distance_modulus

DistanceModulus(Array([10.      ,  8.49485 ,  7.614393], dtype=float32), unit='mag')

<div class="alert alert-block alert-success">
<b>Exercise:</b> Build a DistanceModulus class
</div>

Hint, check out `coordinax.distance.DistanceModulus`

## Unit Systems

If you've used `gala.units` then you're familiar with Astropy-compatible unit systems!

In [42]:
from unxt.unitsystems import unitsystem, galactic, solarsystem

In [43]:
unitsystem?

Signature:       unitsystem(usys: unxt._src.unitsystems.base.AbstractUnitSystem, /) -> unxt._src.unitsystems.base.AbstractUnitSystem
Call signature:  unitsystem(*args, **kw_args)
Type:            Function
String form:     <multiple-dispatch function unitsystem (with 9 registered and 0 pending method(s))>
File:            ~/Documents/Academia/Conferences & Presentations/2025-09-Thunch and Flatiron/Flatiron Presentation/.venv/lib/python3.12/site-packages/unxt/_src/unitsystems/core.py
Docstring:      
Convert a UnitSystem to a UnitSystem.

Examples
--------
>>> from unxt.unitsystems import unitsystem
>>> usys = unitsystem("kpc", "Myr", "Msun", "radian")
>>> usys
unitsystem(kpc, Myr, solMass, rad)

>>> unitsystem(usys) is usys
True

-----------------------------------------------------------------------------------------------------------------------------------------------------------

unitsystem(seq: collections.abc.Sequence[typing.Any], /) -> unxt._src.unitsystems.base.AbstractUnitSyste

In [44]:
usys = unitsystem("kpc", "Gyr", "solMass", "deg")
usys

unitsystem(kpc, Gyr, solMass, deg)

There are also pre-defined unit systems

In [45]:
galactic

unitsystem(kpc, Myr, solMass, rad)

In [46]:
solarsystem

unitsystem(AU, yr, solMass, rad)

The unit systems are useful when working with Quantities.

In [47]:
print(x)
x.decompose(usys)

BareQuantity(weak_i32[], unit='km')


BareQuantity(Array(3.2407792e-15, dtype=float32, weak_type=True), unit='kpc')

In [48]:
x = 1e4 * Quantity([1, 2, 3], 'lyr')
x.uconvert(usys["length"])

Quantity(Array([3.066014, 6.132028, 9.198042], dtype=float32, weak_type=True),
    unit='kpc')

In [49]:
Quantity([1, 2, 3], 'lyr / mas').decompose(usys)

Quantity(Array([1103.765, 2207.53 , 3311.295], dtype=float32, weak_type=True),
    unit='kpc / deg')

## Astropy Compatibility

`unxt.Quantity` can be made from an Astropy Quantity

In [50]:
x = Quantity.from_(u.Quantity(1, "kpc"))
x

Quantity(Array(1., dtype=float32), unit='kpc')

And it can be converted to Astropy

In [51]:
from plum import convert

convert(x, u.Quantity)

<Quantity 1. kpc>

## Differentiation

`unxt` fully integrates with `JAX` differentiation.

In [52]:
def polynomial(
    x: Float[Quantity["area"], "N"],
    a0: Float[Quantity["volume"], ""],
    a1: Float[Quantity["length"], ""],
    a2: Float[Quantity["1/m"], ""],
) -> Float[Quantity["volume"], "N"]:
    return a0 + a1 * x ** 1 + a2 * x ** 2

polynomial

<function __main__.polynomial(x: jaxtyping.Float[Quantity[PhysicalType('area')], 'N'], a0: jaxtyping.Float[Quantity[PhysicalType('volume')], ''], a1: jaxtyping.Float[Quantity[PhysicalType('length')], ''], a2: jaxtyping.Float[Quantity[PhysicalType('wavenumber')], '']) -> jaxtyping.Float[Quantity[PhysicalType('volume')], 'N']>

In [53]:
x = Quantity(1.0, "km2")
a0, a1, a2 = Quantity(1, "km3"), Quantity(2, "km"), Quantity(3, "1/km")

In [54]:
polynomial(x, a0, a1, a2)

Quantity(Array(6., dtype=float32, weak_type=True), unit='km3')

In [55]:
grad_fn = quaxify(jax.grad(polynomial))
grad_fn(x, a0, a1, a2)

Quantity(Array(8., dtype=float32), unit='km')

In [56]:
# Alternatively:

from quaxed import grad as qgrad

qgrad(polynomial)(x, a0, a1, a2)

Quantity(Array(8., dtype=float32), unit='km')